In [0]:
%sh
nslookup priv-testrb-my-endpoint.westeurope.inference.ml.azure.com

Server:		168.63.129.16
Address:	168.63.129.16#53

Non-authoritative answer:
priv-testrb-my-endpoint.westeurope.inference.ml.azure.com	canonical name = priv-testrb-my-endpoint.e2185a76-1fa3-4ad6-a882-f5d6ce95bdb5.inference.westeurope.privatelink.api.azureml.ms.
Name:	priv-testrb-my-endpoint.e2185a76-1fa3-4ad6-a882-f5d6ce95bdb5.inference.westeurope.privatelink.api.azureml.ms
Address: 10.0.0.6



In [0]:
import urllib.request
import json
import os
import ssl

def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

allowSelfSignedHttps(True) # this line is needed if you use self-signed certificate in your scoring service.

data = {
  "input_data": {
    "columns": [
      "age",
      "sex",
      "cp",
      "trestbps",
      "chol",
      "fbs",
      "restecg",
      "thalach",
      "exang",
      "oldpeak",
      "slope",
      "ca",
      "thal"
    ],
    "data": [[63,1,1,145,233,1,2,150,0,2.3,3,0,1],[10,1,1,145,233,1,2,150,0,2.3,3,0,0]]
  }
}

body = str.encode(json.dumps(data))

url = 'https://priv-testrb-my-endpoint.westeurope.inference.ml.azure.com/score'
# Replace this with the primary/secondary key or AMLToken for the endpoint
api_key = '<<your token>>'
if not api_key:
    raise Exception("A key should be provided to invoke the endpoint")

# The azureml-model-deployment header will force the request to go to a specific deployment.
# Remove this header to have the request observe the endpoint traffic rules
headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key), 'azureml-model-deployment': 'priv2-deployment' }

req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)

    result = response.read()
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))

    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(error.read().decode("utf8", 'ignore'))

b'[0, 0]'
